# Crypto usage of qmrExchange

### Import required libraries

In [1]:
import sys
sys.path.append('..')
from main import *  

### Declare basic parameters for the simulation
qmrExchange allows for simulating multiple tickers at once, for statistical arbitrage and high-frequency-trading simulations. In the present case, we simulate 1 week worth of 1 minute data (24/7 trading).


In [2]:
time_interval = 'minute'
tickers = ['XYZ']
episodes = 1000

### Instantiate a Simulator

In [3]:
sim = Simulator(time_unit=time_interval, episodes=episodes)
# create a blockchain with mempool for the exchange
sim.exchange.crypto = True
sim.exchange.create_asset(tickers[0])

print(sim.exchange.blockchain.chain)

### Add trading agents
- We add a naive market maker that creates both buy and sell orders in each period. It quotes buy and sell prices based on the last traded price and the specified spread percentage.
- We add a market taker that randomly buys and sells (based on the defined probabilities) on each period by means of market ordes (hence the word 'taker').

In [4]:

mm = CryptoMarketMaker(name='market_maker', tickers=tickers, aum=1_000, spread_pct=0.005, qty_per_order=4)
sim.add_agent(mm)

mt = CryptoMarketTaker(name='market_taker', tickers=tickers, aum=1_000, prob_buy=.2, prob_sell=.2, qty_per_order=1,seed=42)
sim.add_agent(mt)

print(sim.agents)

[<Agent: market_maker>, <Agent: market_taker>]


### Run the simulation

In [5]:
sim.run()

Retrieve all executed trades of our simulation

In [6]:
# print(sim.exchange.mempool.transactions)
print(sim.transactions.head(20).to_markdown())

| dt                  | ticker    |           fee |   amount | sender       | recipient    | confirmed   | timestamp           |
|:--------------------|:----------|--------------:|---------:|:-------------|:-------------|:------------|:--------------------|
| 1700-01-01 00:00:00 | init_seed |   0           |   0      | init_seed    | init_seed    | True        | NaT                 |
| 1700-01-01 00:00:00 | XYZ       |   0           | 100      | init_seed    | init_seed    | True        | 1700-01-01 00:01:00 |
| 1700-01-01 00:03:00 | XYZ       |   0.001       | 100.25   | market_maker | market_taker | True        | 1700-01-01 00:04:00 |
| 1700-01-01 00:07:00 | XYZ       |   0.000333333 | 100.501  | market_maker | market_taker | True        | 1700-01-01 00:08:00 |
| 1700-01-01 00:10:00 | XYZ       |   1.59734     | 400.997  | market_taker | market_maker | True        | 1700-01-01 00:11:00 |
| 1700-01-01 00:10:00 | XYZ       |   1.59534     |  99      | market_taker | init_seed    | True

Group asset price in fixed 15 Minute OHLCV Bars

In [7]:
df_15min = sim.get_price_bars(ticker=tickers[0],bar_size='15Min')
print(df_15min.head(20).to_markdown())

| dt                  |     open |     high |     low |   close |   volume |
|:--------------------|---------:|---------:|--------:|--------:|---------:|
| 1700-01-01 00:00:00 | 100      | 100.501  | 99      | 99      |        8 |
| 1700-01-01 00:15:00 |  98.7525 |  99.2456 | 98.5032 | 98.5032 |       28 |
| 1700-01-01 00:30:00 |  98.2569 |  98.2569 | 98.01   | 98.01   |       18 |
| 1700-01-01 00:45:00 |  98.2551 |  98.5007 | 98.0082 | 98.0082 |       15 |
| 1700-01-01 01:00:00 |  98.2532 |  98.2532 | 97.2744 | 97.2744 |       17 |
| 1700-01-01 01:15:00 |  97.0312 |  97.0312 | 96.788  | 96.788  |       13 |
| 1700-01-01 01:30:00 |  97.03   |  98.0027 | 97.0294 | 97.7577 |       19 |
| 1700-01-01 01:45:00 |  97.5133 |  98.0002 | 97.5121 | 98.0002 |       16 |
| 1700-01-01 02:00:00 |  98.2452 |  98.7365 | 98.2446 | 98.7365 |        8 |
| 1700-01-01 02:15:00 |  98.4896 |  98.7358 | 98.2428 | 98.2428 |       13 |
| 1700-01-01 02:30:00 |  97.9972 |  98.4878 | 97.9972 | 98.2409 |       15 |

Retrieve a dataframe of an agents holding at each period of time

In [8]:
mt_holdings = sim.get_portfolio_history('market_taker')
mm_holdings = sim.get_portfolio_history('market_maker')

print(mt_holdings.head(20).to_markdown())

|                     |      XYZ |       cash |       aum |
|:--------------------|---------:|-----------:|----------:|
| 1700-01-01 00:00:00 |      0   |     0      |      0    |
| 1700-01-01 00:01:00 |      0   |     0      |      0    |
| 1700-01-01 00:02:00 |      0   |     0      |      0    |
| 1700-01-01 00:03:00 |      0   |     0      |      0    |
| 1700-01-01 00:04:00 |  10050.1 |  -100.251  |   9949.81 |
| 1700-01-01 00:05:00 |  20100.1 |  -200.502  |  19899.6  |
| 1700-01-01 00:06:00 |  30150.2 |  -300.753  |  29849.4  |
| 1700-01-01 00:07:00 |  40300.8 |  -401.004  |  39899.7  |
| 1700-01-01 00:08:00 |  60476.3 |  -601.756  |  59874.6  |
| 1700-01-01 00:09:00 |  80651.9 |  -802.508  |  79849.4  |
| 1700-01-01 00:10:00 |  99321.9 | -1003.26   |  98318.7  |
| 1700-01-01 00:11:00 | 158895   |  -803.014  | 158092    |
| 1700-01-01 00:12:00 | 218468   |  -602.769  | 217865    |
| 1700-01-01 00:13:00 | 278041   |  -402.523  | 277639    |
| 1700-01-01 00:14:00 | 337614   |  -202

### Plot the results

In [9]:
from source.backend.helpers import plot_bars
df_15min = sim.get_price_bars(ticker=tickers[0], bar_size='15Min')
plot_bars(df_15min)

In [10]:
import plotly.express as px
import pandas as pd

df_plot = pd.DataFrame()
df_plot['Market Maker'] = mm_holdings['aum']
df_plot['Market Taker'] = mt_holdings['aum']
fig = px.line(df_plot,labels={'variable':'Agents','value':'Assets Under Management','dt':'Date'})
fig.show()